In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
pip install torch-geometric

In [6]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=32, out_channels=13, num_layers=2).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')

    np.savetxt('/content/drive/MyDrive/PROJECT/train_f1_scores_GAT.txt',train_f1_scores)
    np.savetxt('/content/drive/MyDrive/PROJECT/train_loss_GAT.txt', train_losses)
    np.savetxt('/content/drive/MyDrive/PROJECT/epochs_GAT.txt', epochss)






# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/f1_score_GAT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Using device: cuda
Epoch [1/500], Loss: 2.6427366733551025, Train Acc: 0.0820,train F1-score:0.1011 Val Loss: 1.514008641242981, Val Acc: 0.6787
Epoch [2/500], Loss: 1.7787212133407593, Train Acc: 0.6442,train F1-score:0.5049 Val Loss: 1.1631855964660645, Val Acc: 0.6787
Epoch [3/500], Loss: 1.368759274482727, Train Acc: 0.6406,train F1-score:0.5101 Val Loss: 1.0955708026885986, Val Acc: 0.6667
Epoch [4/500], Loss: 1.2451982498168945, Train Acc: 0.5998,train F1-score:0.5393 Val Loss: 1.0438143014907837, Val Acc: 0.7142
Epoch [5/500], Loss: 1.1677541732788086, Train Acc: 0.6278,train F1-score:0.5706 Val Loss: 0.9630767703056335, Val Acc: 0.7068
Epoch [6/500], Loss: 1.1030761003494263, Train Acc: 0.6573,train F1-score:0.5528 Val Loss: 0.9476293325424194, Val Acc: 0.7075
Epoch [7/500], Loss: 1.0972388982772827, Train Acc: 0.6558,train F1-score:0.5431 Val Loss: 0.8763332962989807, Val Acc: 0.7169
Epoch [8/500], Loss: 1.0285813808441162, Train Acc: 0.6644,train F1-score:0.5717 Val Loss: 0.8

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=8)
decision_tree.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(train_outputs)
val_pred = decision_tree.predict(val_outputs)
test_pred = decision_tree.predict(test_outputs)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
# Save the F1-score to a text file
with open('/content/drive/MyDrive/PROJECT/f1_scores_GAT_DT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Training Accuracy (Decision Tree): 0.934851783620834
Validation Accuracy (Decision Tree): 0.8969210174029452
Test Accuracy (Decision Tree): 0.8947721179624665
Precision: 0.8935, Recall: 0.8948, F1-score: 0.8931
F1-score saved to file.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Concatenate the GIN model output with the original feature matrices
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=7)
decision_tree.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(X_train_combined)
val_pred = decision_tree.predict(X_val_combined)
test_pred = decision_tree.predict(X_test_combined)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_scores_GAT_DT_comb.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")



Training Accuracy (Decision Tree): 0.9266454530229442
Validation Accuracy (Decision Tree): 0.893574297188755
Test Accuracy (Decision Tree): 0.9001340482573726
Precision: 0.8961, Recall: 0.9001, F1-score: 0.8960
F1-score saved to file.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(train_outputs)
val_pred = svm_classifier.predict(val_outputs)
test_pred = svm_classifier.predict(test_outputs)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_scores_GAT_SVM.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Training Accuracy (SVM): 0.9418020432088428
Validation Accuracy (SVM): 0.9049531459170014
Test Accuracy (SVM): 0.9101876675603218
Precision: 0.9104, Recall: 0.9102, F1-score: 0.9099


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score saved to file.


In [26]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Combine the output of the GIN model with the original features
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(X_train_combined)
val_pred = svm_classifier.predict(X_val_combined)
test_pred = svm_classifier.predict(X_test_combined)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_scores_GAT_SVM_COMB.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")



Training Accuracy (SVM): 0.9518506112878915
Validation Accuracy (SVM): 0.9022757697456493
Test Accuracy (SVM): 0.9189008042895442
Precision: 0.9184, Recall: 0.9189, F1-score: 0.9182


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score saved to file.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=8, out_channels=13, num_layers=2).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model_23.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')







# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model_23.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')



from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model_23.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Combine the output of the GIN model with the original features
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(X_train_combined)
val_pred = svm_classifier.predict(X_val_combined)
test_pred = svm_classifier.predict(X_test_combined)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')





Using device: cuda
Epoch [1/500], Loss: 2.8108060359954834, Train Acc: 0.0785,train F1-score:0.0983 Val Loss: 2.0881409645080566, Val Acc: 0.6720
Epoch [2/500], Loss: 2.15169095993042, Train Acc: 0.4686,train F1-score:0.4588 Val Loss: 1.6338529586791992, Val Acc: 0.6787
Epoch [3/500], Loss: 1.7400093078613281, Train Acc: 0.6223,train F1-score:0.5139 Val Loss: 1.273297905921936, Val Acc: 0.6787
Epoch [4/500], Loss: 1.4924780130386353, Train Acc: 0.6389,train F1-score:0.5069 Val Loss: 1.1374655961990356, Val Acc: 0.6787
Epoch [5/500], Loss: 1.4105299711227417, Train Acc: 0.6429,train F1-score:0.5075 Val Loss: 1.0888171195983887, Val Acc: 0.6787
Epoch [6/500], Loss: 1.3879661560058594, Train Acc: 0.6419,train F1-score:0.5074 Val Loss: 1.017858862876892, Val Acc: 0.6787
Epoch [7/500], Loss: 1.2867190837860107, Train Acc: 0.6408,train F1-score:0.5168 Val Loss: 0.984176516532898, Val Acc: 0.6794
Epoch [8/500], Loss: 1.2316173315048218, Train Acc: 0.6214,train F1-score:0.5355 Val Loss: 0.9970

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training Accuracy (SVM): 0.9005191760174175
Validation Accuracy (SVM): 0.8969210174029452
Test Accuracy (SVM): 0.8894101876675603
Precision: 0.8849, Recall: 0.8894, F1-score: 0.8847


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.6, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=128, out_channels=13, num_layers=1).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model1.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')

    np.savetxt('/content/drive/MyDrive/PROJECT/layer1/train_f1_scores_GAT.txt',train_f1_scores)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer1/train_loss_GAT.txt', train_losses)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer1/epochs_GAT.txt', epochss)






# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model1.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/layer1/f1_score_GAT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Using device: cuda
Epoch [1/500], Loss: 2.586740016937256, Train Acc: 0.0724,train F1-score:0.1120 Val Loss: 1.1814968585968018, Val Acc: 0.7008
Epoch [2/500], Loss: 1.3016562461853027, Train Acc: 0.6567,train F1-score:0.5675 Val Loss: 1.1019054651260376, Val Acc: 0.6934
Epoch [3/500], Loss: 1.2367472648620605, Train Acc: 0.6557,train F1-score:0.5326 Val Loss: 0.9842931032180786, Val Acc: 0.7001
Epoch [4/500], Loss: 1.0936131477355957, Train Acc: 0.6715,train F1-score:0.6205 Val Loss: 0.9630354642868042, Val Acc: 0.6888
Epoch [5/500], Loss: 1.0389503240585327, Train Acc: 0.6587,train F1-score:0.6401 Val Loss: 0.843820333480835, Val Acc: 0.7463
Epoch [6/500], Loss: 0.918756902217865, Train Acc: 0.7039,train F1-score:0.6583 Val Loss: 0.8288828134536743, Val Acc: 0.7503
Epoch [7/500], Loss: 0.8875752091407776, Train Acc: 0.7151,train F1-score:0.6582 Val Loss: 0.8249802589416504, Val Acc: 0.7557
Epoch [8/500], Loss: 0.8847577571868896, Train Acc: 0.7223,train F1-score:0.6755 Val Loss: 0.78

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score saved to file.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.6, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=128, out_channels=13, num_layers=3).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model3.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')

    np.savetxt('/content/drive/MyDrive/PROJECT/layer3/train_f1_scores_GAT.txt',train_f1_scores)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer3/train_loss_GAT.txt', train_losses)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer3/epochs_GAT.txt', epochss)






# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model3.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/layer3/f1_score_GAT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Using device: cuda
Epoch [1/500], Loss: 2.652108669281006, Train Acc: 0.0450,train F1-score:0.0674 Val Loss: 18.934040069580078, Val Acc: 0.6787
Epoch [2/500], Loss: 23.0024471282959, Train Acc: 0.6441,train F1-score:0.5047 Val Loss: 2.8651838302612305, Val Acc: 0.1104
Epoch [3/500], Loss: 3.1894607543945312, Train Acc: 0.1241,train F1-score:0.0320 Val Loss: 2.46151065826416, Val Acc: 0.0944
Epoch [4/500], Loss: 2.3673789501190186, Train Acc: 0.1112,train F1-score:0.0364 Val Loss: 2.195315361022949, Val Acc: 0.0937
Epoch [5/500], Loss: 2.233876943588257, Train Acc: 0.1043,train F1-score:0.0247 Val Loss: 2.3424692153930664, Val Acc: 0.0937
Epoch [6/500], Loss: 2.442516565322876, Train Acc: 0.1084,train F1-score:0.0308 Val Loss: 2.241842269897461, Val Acc: 0.0937
Epoch [7/500], Loss: 2.2268707752227783, Train Acc: 0.1068,train F1-score:0.0303 Val Loss: 2.2014408111572266, Val Acc: 0.0937
Epoch [8/500], Loss: 2.1879382133483887, Train Acc: 0.1061,train F1-score:0.0379 Val Loss: 1.88858079

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score saved to file.


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GATNet(torch.nn.Module):
    def __init__(self, in_channels, hidden_dim, out_channels, num_layers):
        super(GATNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_dim, heads=8))
        for _ in range(num_layers - 1):
            self.convs.append(GATConv(hidden_dim * 8, hidden_dim, heads=8))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim * 8, out_channels)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=0.6, training=self.training)
        x = self.fc(x)
        return x

# Define model
model = GATNet(in_channels=X_train.shape[1], hidden_dim=128, out_channels=13, num_layers=4).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []

    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)



        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)
            val_losses.append(val_loss)
            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model4.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f},train F1-score:{train_f1:.4f} Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')

    np.savetxt('/content/drive/MyDrive/PROJECT/layer4/train_f1_scores_GAT.txt',train_f1_scores)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer4/train_loss_GAT.txt', train_losses)
    np.savetxt('/content/drive/MyDrive/PROJECT/layer4/epochs_GAT.txt', epochss)






# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=100)

# Load the best model
model.load_state_dict(torch.load('best_model4.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)

print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')

from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/layer4/f1_score_GAT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")




Using device: cuda
Epoch [1/500], Loss: 2.67704439163208, Train Acc: 0.0536,train F1-score:0.0761 Val Loss: 51.17746353149414, Val Acc: 0.6787
Epoch [2/500], Loss: 65.8505630493164, Train Acc: 0.6441,train F1-score:0.5047 Val Loss: 1.9251996278762817, Val Acc: 0.1104
Epoch [3/500], Loss: 2.332005023956299, Train Acc: 0.1221,train F1-score:0.0395 Val Loss: 2.4797110557556152, Val Acc: 0.1104
Epoch [4/500], Loss: 2.3966269493103027, Train Acc: 0.1183,train F1-score:0.0399 Val Loss: 2.4414517879486084, Val Acc: 0.0529
Epoch [5/500], Loss: 2.404960870742798, Train Acc: 0.1093,train F1-score:0.0397 Val Loss: 2.3092100620269775, Val Acc: 0.1084
Epoch [6/500], Loss: 2.6688804626464844, Train Acc: 0.1089,train F1-score:0.0358 Val Loss: 2.3879055976867676, Val Acc: 0.1044
Epoch [7/500], Loss: 2.325636148452759, Train Acc: 0.1146,train F1-score:0.0435 Val Loss: 2.1743297576904297, Val Acc: 0.1191
Epoch [8/500], Loss: 2.153927803039551, Train Acc: 0.1431,train F1-score:0.0861 Val Loss: 1.66447758

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1-score saved to file.
